In [1]:
%load_ext autoreload
%autoreload 2

# Test partitioning update

In [2]:
from aerospace_chatbot.processing import DocumentProcessor
from aerospace_chatbot.services import EmbeddingService, LLMService, DatabaseService, prompts
from aerospace_chatbot.processing import QAModel
# from langchain_core.documents import Document

test_index={}
# test_index['db_type']='ChromaDB'
test_index['db_type']='Pinecone'
test_index['embedding_service']='OpenAI'
test_index['embedding_model']='text-embedding-3-large'
test_index['llm_service']='OpenAI'
test_index['llm_model']='gpt-4o'

# setup_fixture={}
# setup_fixture['chunk_method']='character_recursive'
chunk_size=400
chunk_overlap=0
batch_size=50

index_name = 'text-embedding-3-large-test'
rag_type = 'Standard'

# Load environment variables
from dotenv import load_dotenv
load_dotenv(override=True)
# Set LOCAL_DB_PATH environment variable
# os.environ['LOCAL_DB_PATH'] = os.path.abspath('.')

# Initialize logger
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

test_prompt='How does a thermal knife function in a cable based hold down release mechanism?'

In [3]:
# Initialize services
embedding_service = EmbeddingService(
    model_service=test_index['embedding_service'],
    model=test_index['embedding_model']
)

llm_service = LLMService(
    model_service=test_index['llm_service'],
    model=test_index['llm_model'],
)

doc_processor = DocumentProcessor(
    embedding_service=embedding_service,
    rag_type=rag_type,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

# Initialize database service
db_service = DatabaseService(
    db_type=test_index['db_type'],
    index_name=index_name,
    rag_type=rag_type,
    embedding_service=embedding_service,
)

In [4]:
bucket_name = 'processing-pdfs'
docs = DocumentProcessor.list_bucket_pdfs(bucket_name)
docs

INFO:aerospace_chatbot.processing.documents:Number of PDFs found: 2
INFO:aerospace_chatbot.processing.documents:PDFs found: ['gs://processing-pdfs/1999_christiansen_reocr.pdf', 'gs://processing-pdfs/1999_cremers_reocr.pdf']


['gs://processing-pdfs/1999_christiansen_reocr.pdf',
 'gs://processing-pdfs/1999_cremers_reocr.pdf']

In [5]:
# partitioned_docs = doc_processor.load_and_partition_documents(docs,partition_by_api=False, upload_bucket=bucket_name)
# partitioned_docs

In [6]:
# chunk_obj, output_paths = doc_processor.chunk_documents(partitioned_docs)
# chunk_obj.chunk_convert(destination_type=Document)

In [7]:
try:
    db_service.initialize_database(clear=False)
except ValueError as e:
    print(f"Database initialization failed: {str(e)}")
    print(e)
    raise e

INFO:aerospace_chatbot.services.database:Validating index text-embedding-3-large-test and RAG type Standard
INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Users/danmueller/Documents/GitHub/aerospace_chatbot/.venv/lib/python3.11/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
INFO:aerospace_chatbot.services.database:Pinecone index text-embedding-3-large-test found, not creating. Will be initialized with existing index.


In [8]:
# db_service.index_data(chunk_obj)

In [9]:
qa_model = QAModel(
    db_service=db_service,
    llm_service=llm_service,
    k=8
)

/Users/danmueller/Documents/GitHub/aerospace_chatbot/src/aerospace_chatbot/processing/queries.py:47: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(


In [10]:
# qa_model.query(test_prompt)

In [11]:
# print(qa_model.result[-1]['references'])
# print(qa_model.sources[-1])
# print(qa_model.scores[-1])


In [12]:
# print(qa_model.ai_response)

# Langgraph

Run above section first


In [13]:
from langchain_core.messages import SystemMessage, RemoveMessage
from langchain.prompts import HumanMessagePromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.documents import Document
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

from typing_extensions import List
from typing import List, Literal
from pydantic import BaseModel, Field, field_validator, ValidationInfo
import re

from aerospace_chatbot.services.prompts import CHATBOT_SYSTEM_PROMPT, QA_PROMPT, SUMMARIZE_TEXT

In [14]:
retriever = db_service.retriever
llm = llm_service.get_llm()
memory = MemorySaver()
config = {"configurable": {"thread_id": "1"}}

In [63]:
class InLineCitationsResponse(BaseModel):
    content: str = Field(description="The main content of the response with in-line citations.")
    citations: List[str] = Field(description="List of extracted source IDs from the response.")

    # Validator to ensure citations follow the <source id="#"> format
    @field_validator('content')
    def validate_citations(cls, v):
        # Regex pattern to match <source id="1">, <source id="2">, etc.
        pattern = r'<source id="(\d+)">'
        matches = re.findall(pattern, v)
        
        # Raise error if no citations are found or formatting is incorrect
        if not matches:
            raise ValueError('No valid source tags found. Expected format: <source id="1">')

        return v

    # Validator to extract and populate citations from content
    @field_validator('citations', mode='before')
    def extract_citations(cls, v, info: ValidationInfo):
        # Access content field from the model
        content = info.data.get('content', '')
        pattern = r'<source id="(\d+)">'
        extracted = re.findall(pattern, content)
        
        if not extracted:
            raise ValueError("No citations found in the content. Ensure sources are cited correctly.")
        
        return extracted

In [64]:
valid_response = InLineCitationsResponse(content="""
The actuator was tested under high pressure <source id="1">. 
Material properties were measured over 50 cycles <source id="2">.
Thermal resistance improved by 30% <source id="3">.
""",
citations=["1", "2", "3"]
)

print(valid_response)

content='\nThe actuator was tested under high pressure <source id="1">. \nMaterial properties were measured over 50 cycles <source id="2">.\nThermal resistance improved by 30% <source id="3">.\n' citations=['1', '2', '3']


In [65]:
try:
    invalid_response = InLineCitationsResponse(content="""
    The actuator was tested under high pressure [1]. 
    Material properties were measured under load <source id="x">.
    """,
    citations=["1", "x"]
    )
except:
    print(f"Validation failed")

Validation failed


In [67]:
retrieved_docs = retriever.invoke(test_prompt)

# Add context to the prompt
# TODO update this to use the doc.id. Do some testing to check this works.
docs_content=""
for i, doc in enumerate(retrieved_docs[0]):
    # Source IDs in the order they show in in the array. Indexed from 0.
    docs_content += f"Source ID: {i}\n{doc.page_content}\n\n"

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [82]:
# Define the output parser with the expected Pydantic model
output_parser = PydanticOutputParser(pydantic_object=InLineCitationsResponse)

CHATBOT_SYSTEM_PROMPT=SystemMessage(content=
"""
Your name is **Aerospace Chatbot**, a specialized assistant for flight hardware design and analysis in aerospace engineering.

Use only the **Sources and Context** from the **Reference Documents** provided to answer the **User Question**. Do not use outside knowledge, and strictly follow these rules:

---

### **Rules**:
1. **Answer only based on the provided Sources and Context.**  
   - If the information is not available in the Sources and Context, respond with:  
     *"I don’t know the answer to that based on the information provided. You might consider rephrasing your question or asking about a related topic."*

2. **Do not make up or infer answers.**

3. **Provide responses in English only** and format them using **Markdown** for clarity.

4. **Cite Sources in context** using the exact format `<source id="#">`:  
   - `#` – Represents the numerical order of the source as provided in the Sources and Context.  
   - **The `source` tag must be present for every source referenced in the response.**  
   - **Do not add, omit, or modify any part of the citation format.**  
   
   **Examples (Correct):**  
   > The actuator was tested under extreme conditions <source id="1">.  
   > A secondary material exhibited increased yield strength <source id="2">.  
   > Additional research confirmed thermal properties <source id="3">.  

   **Examples (Incorrect – Must Be Rejected):**  
   > Testing yielded higher efficiency [1] (Incorrect bracket format)  
   > <source id="1" > (Extra space after `id`)  
   > <source id="a"> (Non-numeric ID)  
   > <source id="1,2"> (Multiple IDs in one tag – invalid)  

5. **Every sentence or paragraph that uses a source must cite it with the format `<source id="#">`.**  
   - **Do not group multiple sources into a single tag.** Each source must have its own, clearly separated citation.  
   - For example:  
     > The actuator uses a reinforced composite structure <source id="1">. This design was validated through multiple tests <source id="2">.  

6. **Validation Requirement:**  
   - If the response contains references without the exact `<source id="#">` format, the response must be flagged or rejected.  
   - Every source used must have a corresponding citation in the response. **No source should be referenced without explicit citation.**  

7. **Suggest related or alternative questions** if applicable, to help the user find relevant information within the corpus.
""")

QA_PROMPT_TEST=PromptTemplate(
    template=
"""
Your name is **Aerospace Chatbot**, a specialized assistant for flight hardware design and analysis in aerospace engineering.

Use only the **Sources and Context** from the **Reference Documents** provided to answer the **User Question**. Do not use outside knowledge, and strictly follow these rules:

---

### **Rules**:
1. **Answer only based on the provided Sources and Context.**  
   - If the information is not available in the Sources and Context, respond with:  
     *"I don’t know the answer to that based on the information provided. You might consider rephrasing your question or asking about a related topic."*

2. **Do not make up or infer answers.**

3. **Provide responses in English only** and format them using **Markdown** for clarity.

4. **Cite Sources in context** using the exact format `<source id="#">`:  
   - `#` – Represents the numerical order of the source as provided in the Sources and Context.  
   - **The `source` tag must be present for every source referenced in the response.**  
   - **Do not add, omit, or modify any part of the citation format.**  
   
   **Examples (Correct):**  
   > The actuator was tested under extreme conditions <source id="1">.  
   > A secondary material exhibited increased yield strength <source id="2">.  
   > Additional research confirmed thermal properties <source id="3">.  

   **Examples (Incorrect – Must Be Rejected):**  
   > Testing yielded higher efficiency [1] (Incorrect bracket format)  
   > <source id="1" > (Extra space after `id`)  
   > <source id="a"> (Non-numeric ID)  
   > <source id="1,2"> (Multiple IDs in one tag – invalid)  

5. **Every sentence or paragraph that uses a source must cite it with the format `<source id="#">`.**  
   - **Do not group multiple sources into a single tag.** Each source must have its own, clearly separated citation.  
   - For example:  
     > The actuator uses a reinforced composite structure <source id="1">. This design was validated through multiple tests <source id="2">.  

6. **Validation Requirement:**  
   - If the response contains references without the exact `<source id="#">` format, the response must be flagged or rejected.  
   - Every source used must have a corresponding citation in the response. **No source should be referenced without explicit citation.**  

7. **Suggest related or alternative questions** if applicable, to help the user find relevant information within the corpus.

---
**Sources and Context from Reference Documents**:
{context}
---

---
**User Question**:
{question}
---

---
{format_instructions}
---
""",
    input_variables=["context", "question"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

QA_PROMPT=HumanMessagePromptTemplate.from_template(
    template=
"""
---
**Sources and Context from Reference Documents**:
{context}
---

---
**User Question**:
{question}
---

---
{format_instructions}
---
""",
    input_variables=["context", "question"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)


In [84]:
prompt = QA_PROMPT_TEST.format(context=docs_content, question=test_prompt)
raw_output = llm(prompt)
print(raw_output)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


content='```json\n{\n  "content": "The thermal knife in a cable-based hold down release mechanism functions by using a heater plate that is pushed through a wire bundle by a compression spring. This action cuts the Dyneema wire bundle, which is responsible for clamping the two parts of the Reel cable element together. Once the wire bundle is cut, the upper part of the Reel is released, allowing it to deploy along with the application. Both the prime and redundant thermal knives operate along the same center line, ensuring that their heater plates make contact at their cutting edges after cutting the wire bundle. To ensure head-on contact, each thermal knife is attached to the holddown bracket at an angle of approximately 8° <source id=\\"1\\"><source id=\\"2\\"><source id=\\"4\\">.",\n  "citations": [\n    "1",\n    "2",\n    "4"\n  ]\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 185, 'prompt_tokens': 1379, 'total_tokens': 1564, 'c

In [85]:
parsed_response = output_parser.parse(raw_output.content)
print(parsed_response)

content='The thermal knife in a cable-based hold down release mechanism functions by using a heater plate that is pushed through a wire bundle by a compression spring. This action cuts the Dyneema wire bundle, which is responsible for clamping the two parts of the Reel cable element together. Once the wire bundle is cut, the upper part of the Reel is released, allowing it to deploy along with the application. Both the prime and redundant thermal knives operate along the same center line, ensuring that their heater plates make contact at their cutting edges after cutting the wire bundle. To ensure head-on contact, each thermal knife is attached to the holddown bracket at an angle of approximately 8° <source id="1"><source id="2"><source id="4">.' citations=['1', '2', '4']


In [15]:
class State(MessagesState):
    context: List[Document]
    cited_sources: List[str]
    summary: str

# Define application steps
def retrieve(state: State):
    """
    Retrieve the documents from the database.
    """
    logger.info(f"Node: retrieve")

    retrieved_docs = retriever.invoke(state["messages"][-1].content)
    
    # TODO: Grow context with messages, include cited sources
    return {"context": retrieved_docs}

def generate_w_context(state: State):
    """
    Call the model with the prompt with context.
    """
    logger.info(f"Node: generate_w_context")

    # Get the summary
    # TODO add conversation modes
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [CHATBOT_SYSTEM_PROMPT] + [SystemMessage(content=system_message)] + state["messages"]
        # logger.info(f"Messages with system prompt (w/summary): {messages}")
    else:
        messages = [CHATBOT_SYSTEM_PROMPT] + state["messages"]
        # logger.info(f"Messages with system prompt (w/o summary): {messages}")

    # Add context to the prompt
    docs_content=""
    for i, doc in enumerate(state["context"]):
        # Source IDs in the order they show in in the array. Indexed from 0.
        docs_content += f"Source ID: {i}\n{doc.page_content}\n\n"

    # Prompt with context and pydantic output parser
    prompt_with_context = QA_PROMPT.format(
        context=docs_content,
        question=state["messages"][-1].content, 
    )
    # Replace the last message (user question) with the prompt with context, return LLM response
    messages[-1] = prompt_with_context 
    response = llm.invoke(messages)
    logger.info(f"Response from LLM: {response}")

    # Parse the response. This will return a InLineCitationsResponse object. This object has two fields: content and citations.
    # For now, replace the last message with the content of the parsed and validated response. 
    # Add the citations to the state, only for the last message.
    parsed_response = output_parser.parse(response.content)
    logger.info(f"Parsed response: {parsed_response}")
    response.content = parsed_response.content
    logger.info(f"Response after parsing: {response}")

    # Update the state messages with the messages updated in this node.
    state["messages"] = messages
    logger.info(f"Messages after generate_w_context: {state['messages']}")
    return {"messages": [response]}

def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """
    Define the logic for determining whether to end or summarize the conversation
    """
    logger.info(f"Node: should_continue")

    # If there are more than six messages, then we summarize the conversation
    messages = state["messages"]
    if len(messages) > 6:
        logger.info(f"Summarizing conversation")
        return "summarize_conversation"
    
    # Otherwise just end
    logger.info(f"Ending conversation")
    # logger.info(f"Messages before ending: {messages}")
    return END

def summarize_conversation(state: State):
    """
    Summarize the conversation
    """
    logger.info(f"Node: summarize_conversation")

    summary = state.get("summary", "")
    if summary:
        # If a summary already exists, extend it
        summary_message = SUMMARIZE_TEXT.format(
            summary=summary,
            augment="Extend the summary by taking into account the new messages above."
        )
    else:
        # If no summary exists, create one
        summary_text="""---\n**Conversation Summary to Date**:\n{summary}\n---"""
        summary_message = SUMMARIZE_TEXT.format(
            summary=summary_text,
            augment="Create a summary of the conversation above."
        )

    messages = state["messages"] + [summary_message]
    response = llm.invoke(messages)

    # Prune messages. This deletes all but the last two messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

In [16]:
# Compile application and test
workflow = StateGraph(State)

# Define the conversation node and the summarize node
workflow.add_node("retrieve", retrieve) 
workflow.add_node("generate_w_context", generate_w_context)
workflow.add_node("summarize_conversation", summarize_conversation)

workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "generate_w_context")

# We now add a conditional edge
workflow.add_conditional_edges(
    # Define the start node. We use `generate_w_context`. This means these are the edges taken after the `conversation` node is called.
    "generate_w_context",
    # Next, pass in the function that will determine which node is called next.
    should_continue,
)

# Add a normal edge from `summarize_conversation` to END. This means that after `summarize_conversation` is called, we end.
workflow.add_edge("summarize_conversation", END)

app = workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# result = graph.invoke({"question": test_prompt}, config=config)

prompt = 'My name is Dan. Please tell me about some interesting mecanism designs.'
result = app.invoke({"messages": [("human", prompt)]}, config)
for message in result['messages']:
    message.pretty_print()

In [35]:
# prompt = 'How have these mecahnisms been tested?'
# result = app.invoke({"messages": [("human", prompt)]}, config)
# for message in result['messages']:
#     message.pretty_print()

In [36]:
# prompt = 'How old are you?'
# result = app.invoke({"messages": [("human", prompt)]}, config)
# for message in result['messages']:
#     message.pretty_print()

In [37]:
# prompt = 'What are some lessons learned about these mechanisms?'
# result = app.invoke({"messages": [("human", prompt)]}, config)
# for message in result['messages']:
#     message.pretty_print()

In [38]:
# result

In [39]:
# prompt = 'What are some problems that have occurred?'
# result = app.invoke({"messages": [("human", prompt)]}, config)
# for message in result['messages']:
#     message.pretty_print()